In [1]:
# YOU DON'T NEED TO READ THIS SECTION UNLESS YOU WANT TO KNOW THE IMPLEMENT DETAILS OF FIGURES
import numpy
import matplotlib
import ipywidgets
%matplotlib inline

# The Introduction

[Bitcoin](https://bitcoin.org/), [ethereum](https://ethereum.org/) and other blockchains are developed so quickly during 2010s. Blockchains provide a decentrilized and auditable infrastructure to the users, allowing all members to maintain their own copies of the same ledger and ensure that the copies are all synchronized. Blockchains can be used as a ledger, a global computer or other state machines and it drives many on-chain applications.

One of the key challenges of this decentralized design is incentives. The incentive model of a blockchain is to calculating the system income (including inflation and transaction fee) and design the redistribution of the income. Another article of mine can be read (TODO: add link here) if you want to know more about the whole picture of fundamental mechanism of blockchains. Simply speaking, ignoring the difference among various consensus algrithms, the responsible participants of on-chain governance should be motivated and [bitcoin](https://bitcoin.org/), [ethereum](https://ethereum.org/), [eos](https://eos.io/) and other blockchains proposed various good solutions.

Actually on-chain behavior should be rewarded by on-chain tokens, which means, the incentive model is based on a basic token model of a blockchain. Nowadays, most of blockchains use single token model. The only token in the single token model may be used as a governance token as well as a utility token and more or less, it worked pretty well.

Blockchains like [neo](https://neo.org/) and [ontology](https://ont.io/) use a dual token model. One is governance token without inflation while the other is utility token with inflation. With inflation people may spend it while without inflation people may hold it.

In this article, the problems in the dual token model will be discussed and a comparasion with single token model will be introduced. In the end, an interesting dual token economic model will be proposed.

*In the rest of this article, [neo](https://neo.org/) will be used as an example of explaination. We will follow the design of neo blockchain and use NEO to represent the governance token and GAS to represent the utility token.*

*In this article, it is focused on the token model instead of incentive model. So the detail of reward redistribution is ommited. For more detail about incentive model, see another article of mine (TODO: add link here).*

# The Problem

As an example, current token model of [neo](http://neo.org/) will be shown below:

> There are 2 kinds of native token defined in neo system: NEO and GAS.
> 
> NEO is Governing Token. Neo holders can take part in NEO network management, including voting for consensus nodes, network parameter modification, etc. NEO's total amount is 100 million. Its minimum unit is 1 and can not be divided. It's registered in Genesis block, and stored in standby validators' muti-signature addresses.
>
> GAS is utility token, or network fuel token. Operation costs and incentive for consensus node in NEO network are paid in GAS. GAS's total amount is also 100 million and can be divided to 0.00000001 unit. Gas is registered in Genesis block and issued afterwards as block incentive for NEO holders.
> A new block is generated about every 15 seconds according to system design. Thus approximately 2 million new blocks are created every year. 100 million GAS is issued gradually in 22 years according to a attenuation algorithm. No GAS is issued afterwards.

PoS and BFT like blockchains need governance mechanism to make on-chain decision and incentive mechanism to reward responsible governance participants. However, the implements detail of governance mechanism and incentive mechanism will not be introduced here. Some of the NEO holders are governance participants because NEO is the governance token. GAS can only be paid as transaction fees.

In the version 2 of [neo](http://neo.org/), the inflation GAS is distributed to NEO holders, which makes NEO holders too lazy to participate in on-chain governance because NEO holders get the same GAS whether they participate or not. Thus, in the version 3 of [neo](http://neo.org/), the inflation GAS is only distributed to governance participants as a reward.

In this article, governance participants is used instead of NEO holders. You should keep in mind that governace partcipants are not equivalent to NEO holders.

However, the problem is that, the low demand of GAS makes GAS valueless, and easily been controlled in the market and exchanges by attackers. Currently there are not enough on-chain transactions which need a lot of GAS. Almost the only thing that a governance participant can to when it obtains a GAS is to sell it. Then, more want to sell it but less want to buy it. GAS becomes valueless. As a result, governance participants can not get enough reward if the price of GAS is too low.

In this article, solution will be proposed to solve the GAS dilemma.

# The Solution

*Only the principles are statemented in this section. All the incentive details will be ommited here.*

In [2]:
# YOU DON'T NEED TO READ THIS SECTION UNLESS YOU WANT TO KNOW THE IMPLEMENT DETAILS OF FIGURES
class solution:
    S=ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
    T=ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)
    K=ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)
    @classmethod
    def decorator(self):
        return ipywidgets.interact(
            S=self.S,
            T=self.T,
            K=self.K,
        )
    def __init__(self, S, T, K):
        self.S = S
        self.T = T
        self.K = K
    class r:
        def __init__(self, context):
            self.context = context
        def proportion(self, x):
            return self.context.S * numpy.arctan(self.context.K * x) + self.context.T
        def gn(self, g, n):
            return self.proportion(g/n)
    class w:
        def __init__(self, context):
            self.context = context
        def nr(self, n, r):
            return n * r
        def gn(self, g, n):
            return n * solution.r(self.context).gn(g, n)

## No Inflation on NEO

As a respect to the original design, NEO can be kept with no inflation.

## Permanent Inflation on GAS

Inflation GAS is used to incentive governance participants, permanent inflation on GAS is proposed to make the solution works in the long run.

## Redistribution of Inflation GAS

The reward weight rate $r$ is defined in the following expression and it is used in the redistribution of inflation of GAS to evaluate how much weight per NEO the specific governance participant has. The specific governance participant owns $g$ GAS and $n$ NEO. Then an $\arctan$ function is attached to the proportion of GAS and NEO held by the participant and the proportion will be converged between $0$ and $\frac{\pi}{2}$. At last, the additional reward weight rate $S$, the minimal reward weight rate $T$ and the additional reward weight speed $K$ are attached as a linear scale to adjust the basic reward rate and addtional reward rate.

$$
r = S \arctan{\frac{g}{n} K} + T
$$

The following figure shows the relation between reward weight rate $r$ and proportion of GAS and NEO $\frac{g}{n}$. Obviously the reward rate grows when the proportion of GAS and NEO grows. The growth rate of reward weight rate $r$ decreases when proportion of GAS and NEO $\frac{g}{n}$ increases.

In [3]:
@solution.decorator()
def draw(S, T, K):
    c = solution(S, T, K)
    x = numpy.arange(0,100,0.001)
    y = solution.r(c).proportion(x)
    matplotlib.pyplot.xlabel('g/n')
    matplotlib.pyplot.ylabel('r')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

It would be more clear if logarithmic scale is used to plot the relation between reward weight rate $r$ and proportion of GAS and NEO $\frac{g}{n}$ in the follwing figure. It is pretty hard to increase reward weight rate $r$ a bit if proportion of GAS and NEO $\frac{g}{n}$ is enough large.

In [4]:
@solution.decorator()
def draw(S, T, K):
    c = solution(S, T, K)
    x = numpy.arange(-8,8,0.001)
    y = solution.r(c).proportion(10**x)
    matplotlib.pyplot.xlabel('log(g/n)')
    matplotlib.pyplot.ylabel('r')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

The reward weight $w$ is defined as the production of the number of NEO $n$ held by the participant and its reward weight rate $r$.

$$
w = n r
$$

The following 3D figure shows the relation of the number of NEO $n$ and the number of GAS $g$ held by the participant and its reward weight rate $r$.

In [5]:
@solution.decorator()
def draw(S, T, K):
    c = solution(S, T, K)
    x = numpy.arange(1,10,1)
    y = numpy.arange(0.1,100,0.1)
    x, y = numpy.meshgrid(x, y)
    z = solution.w(c).gn(y, x)
    matplotlib.pyplot.gca(projection='3d').plot_surface(x, y, z)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

As a result, the reward $e$ in GAS can be counted as the following expression where $I$ is the amount of inflation GAS, $V$ is all the governance participants and $w_i$ is the reward weight of governance participant $i$.

$$
e = \frac{I w}{\sum_{i \in V}{w_i}}
$$

# The Analysis

## Reward Weight Rate

Reward weight rate $r$ is designed to distribute the inflation GAS. The $\arctan$ function provides pretty good features for the distribution weight. Governance participants will get the basic reward weight if it doesn't hold any GAS, which means, the NEO holder will be incentived if it participated in the on-chain governance. The additional reward is limitted even if the governance participant hold large amount of GAS. However, over all, participants should know:

More NEO held, more GAS rewarded.

$$
\frac{\partial e}{\partial n} \gt 0
$$

More GAS held, more GAS rewarded.

$$\frac{\partial e}{\partial g} \gt 0$$

It is harmless for participants to spend a bit GAS if it need to because only a bit GAS will not decrease much of your reward weight rate if the governance participant hold enough GAS.

## Retial Investor

Alice is a retial investor and she has totally some money for investment. She likes [neo](https://neo.org/) blockchain and she want to spend all her money to buy the native tokens on [neo](https://neo.org/) blockchain, a.k.a NEO and GAS. How many NEO coins should she buy and how many GAS coins should she buy? Which strategy should she use?

By assuming Alice has $A$ dollars and $N$ is the price of NEO in dollar and $G$ is the price of GAS in dollar, the following expression is statemented because Alice spend all her money to buy NEO and GAS.

In [9]:
# YOU DON'T NEED TO READ THIS SECTION UNLESS YOU WANT TO KNOW THE IMPLEMENT DETAILS OF FIGURES
class retial_investor(solution):
    A=ipywidgets.widgets.FloatSlider(min=0, max=1000, step=10, value=100)
    G=ipywidgets.widget.ACKTR('Mls.FloatSlider(min=0, max=100, step=1, value=2)
    N=ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=10)
    I = ipywidgets.widgets.FloatSlider(min=10**6, max=10**8, step=10**6, value=10**7)
    U = ipywidgets.widgets.FloatSlider(min=10**8, max=10**8, step=10**8, value=10**8)
    R = ipywidgets.widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25)
    W = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.01, value=0.4)
    @classmethod
    def decorator(self):
        return ipywidgets.interact(
            S=self.S,
            T=self.T,
            K=self.K,
            A=self.A,
            G=self.G,
            N=self.N,
            I=self.I,
            U=self.U,
            R=self.R,
            W=self.W,
        )
    def __init__(self, S, T, K, A, G, N, I, U, R, W):
        super().__init__(S, T, K)
        self.A = A
        self.G = G
        self.N = N
        self.I = I
        self.U = U
        self.R = R
        self.W = W
    class g:
        def __init__(self, context):
            self.context = context
        def n(self, n):
            return (self.context.A - n * self.context.N) / self.context.G
    class r:
        def __init__(self, context):
            self.context = context
        def n(self, n):
            g = retial_investor.g(self.context).n(n)
            return solution.r(self.context).gn(g, n)
    class w:
        def __init__(self, context):
            self.context = context
        def n(self, n):
            g = retial_investor.g(self.context).n(n)
            return solution.w(self.context).gn(g, n)
    class e:
        def __init__(self, context):
            self.context = context
        def n(self, n):
            return retial_investor.w(self.context).n(n) * self.context.I / self.context.U / self.context.R / self.context.W
    class p:
        def __init__(self, context):
            self.context = context
        def n(self, n):
            return retial_investor.e(self.context).n(n) * self.context.G / self.context.A

SyntaxError: EOL while scanning string literal (<ipython-input-9-6e290785eae3>, line 4)

$$
A = n N + g G
$$

Transform it into another form. The relation between $g$ and $n$ is shown:

$$
g = \frac{A - n N}{G}
$$

It is obvious that such relation is linear.

In [ ]:
@retial_investor.decorator()
def draw(S, T, K, A, G, N, I, U, R, W):
    c = retial_investor(S, T, K, A, G, N, I, U, R, W)
    x = numpy.arange(0.001,A/N,0.001)
    y = retial_investor.g(c).n(x)
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('g')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

Then the relation between reward weight rate $r$ and number of NEO $n$ held by Alice can be calculated.

$$
r = S \arctan{\frac{A - n N}{n G} K} + T
$$

The more NEO Alice buys, the less GAS Alice buys, then, the less reward weight rate $r$.

In [ ]:
@retial_investor.decorator()
def draw(S, T, K, A, G, N, I, U, R, W):
    c = retial_investor(S, T, K, A, G, N, I, U, R, W)
    x = numpy.arange(0.001,A/N,0.001)
    y = retial_investor.r(c).n(x)
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('r')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

The reward weight $w$ can be evaluated by multiplying the number of NEO $n$ held by Alice.

$$
w = n S \arctan{\frac{A - n N}{n G} K} + n T
$$

The following figure shows the relation between reward weight $w$ and number of NEO $n$ bought by Alice. At first, reward weight $w$ grows when $n$ grows but later reward weight $w$ goes down.

In [ ]:
@retial_investor.decorator()
def draw(S, T, K, A, G, N, I, U, R, W):
    c = retial_investor(S, T, K, A, G, N, I, U, R, W)
    x = numpy.arange(0.001,A/N,0.001)
    y = retial_investor.w(c).n(x)
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('w')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

By assuming the total supply of NEO is $U$, the rate of $NEO$ which participated in blockchain governance is $R$ and the average reward weight per NEO is $W$, we have $\sum_{i \in V}{w_i} = U R W$. Then the relation between reward $e$ in GAS and reward weight $w$ can be evaluated as the following expression:

$$
e = \frac{w I}{U R W}
$$

The relation between reward $e$ in GAS and amount of NEO $n$ bought by Alice can be shown in the following figure:

In [ ]:
@retial_investor.decorator()
def draw(S, T, K, A, G, N, I, U, R, W):
    c = retial_investor(S, T, K, A, G, N, I, U, R, W)
    x = numpy.arange(0.001,A/N,0.001)
    y = retial_investor.e(c).n(x)
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('e')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

Then the rate of profit $p$ in dollar can be couted, with the assumption that the price of GAS in dollar will not changed after the inflation:

$$
p = \frac{e G}{A}
$$

And the relation between rate of profit $p$ and amount of NEO $n$ bought by Alice can be drawn:

In [ ]:
@retial_investor.decorator()
def draw(S, T, K, A, G, N, I, U, R, W):
    c = retial_investor(S, T, K, A, G, N, I, U, R, W)
    x = numpy.arange(0.001,A/N,0.001)
    y = retial_investor.p(c).n(x)
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('p')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

As a result, the best strategy for Alice is to buy $n$ NEO to reach the highest rate of profit $p$.

## Exchange Rate

The solution makes a soft exchange rate for NEO and GAS.

$$
N \approx x G
$$

The following expression shows the condition of no need to buy GAS, which is, the maximum price of GAS:

$$
\frac{\partial e}{\partial n} \Big | _{n=\frac{A}{N}} = 0
$$

The following expression show the condition of the balance demand and supply where $P$ is the total supply of NEO and $Q$ is the total supply of GAS:

$$
\begin{cases}
\frac{\partial e}{\partial g} = 0 \\
\frac{g}{n} = \frac{Q}{P}\\
\end{cases}
$$

As a result, NEO will hold the price of GAS when the price of GAS goes down, and GAS will hold the price of NEO when the price of NEO goes down because of the rational players.

# The Conclusion

With the proposed solution in this article, the advantages of dual token economic model can be really activated. We hope the designers of blockchains can be inspired from this article and make their blockchain more sustainable.

# The Appendix

## Notation

- $A$: the total money in dollar owned by the specific participant of neo governance
- $e$: reward in GAS
- $G$: price of GAS in dollar
- $g$: amount of GAS held by the specific participant of neo governance
- $I$: amount of inflation GAS
- $N$: price of NEO in dollar
- $n$: amount of NEO held by the specific participant of neo governance
- $P$: total supply of NEO
- $p$: rate of profit in dollar
- $Q$: total supply of GAS
- $r$: reward weight rate
- $S$: additional reward weight rate
- $T$: minimal reward weight rate
- $V$: the set of all governance participant
- $w$: reward weight
- $x$: exchange rate of NEO and GAS

@vang1ong7ang